# model

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Conv1D, Flatten

2024-11-03 11:33:59.526127: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_pickle('../data/data.pkl') 

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17520 entries, 2022-01-01 01:00:00 to 2024-01-01 00:00:00
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   solar          17520 non-null  int64  
 1   wind_offshore  17520 non-null  int64  
 2   wind_onshore   17520 non-null  int64  
 3   total_load     17520 non-null  int64  
 4   price          17520 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 821.2 KB


In [16]:
# Define function for train-test split to retain all hourly data points in the last 7 days
def train_test_split(data):
    test_indices = data.index.to_series().groupby([data.index.year, data.index.month]).apply(lambda x: x[-24*7:])
    test_data = data.loc[test_indices]
    train_data = data.drop(test_indices)
    return train_data, test_data

# Perform the train-test split
train_data, test_data = train_test_split(data)

In [32]:
# 2. Create Sequences for Time Series Data
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)
        y.append(data['price'].iloc[i+seq_length])
    return np.array(X), np.array(y)

# Set sequence length (e.g., 24 for daily pattern)
seq_length = 24
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

## Model

### LTSM

In [26]:
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, activation='relu', input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

### CNN

In [28]:
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        Flatten(),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

### GRU

In [30]:
def create_gru_model(input_shape):
    model = Sequential([
        GRU(50, activation='relu', input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mae')
    return model


## Training

In [36]:
# Set input shape based on X_train
input_shape = (X_train.shape[1], X_train.shape[2])

In [37]:
lstm_model = create_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 4494.5557 - val_loss: 663.5211
Epoch 2/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 886.7620 - val_loss: 192.8864
Epoch 3/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 167.2725 - val_loss: 109.5191
Epoch 4/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 112.1403 - val_loss: 113.9066
Epoch 5/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 106.9638 - val_loss: 106.9782
Epoch 6/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 96.5006 - val_loss: 100.8383
Epoch 7/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 97.3009 - val_loss: 100.3692
Epoch 8/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 98.7017 - val_loss: 93.9220
Epoch 9/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 98.3386 - val_loss: 108.6389
Epoch 10/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 93.3669 - val_loss: 107.4423


In [38]:
# Train CNN
cnn_model = create_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 966.0616 - val_loss: 192.0738
Epoch 2/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 222.0738 - val_loss: 179.1259
Epoch 3/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 164.7022 - val_loss: 158.3282
Epoch 4/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 135.3566 - val_loss: 88.5357
Epoch 5/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 123.2223 - val_loss: 299.4613
Epoch 6/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 138.3978 - val_loss: 94.4918
Epoch 7/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 103.0097 - val_loss: 98.6727
Epoch 8/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 105.7189 - val_loss: 103.5996
Epoch 9/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 74.7016 - val_loss: 59.8840
Epoch 10/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - loss: 78.6274 - val_loss: 46.2326


In [39]:
# Train GRU
gru_model = create_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 338.8062 - val_loss: 68.4104
Epoch 2/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 119.8285 - val_loss: 33.2506
Epoch 3/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 50.1622 - val_loss: 56.8667
Epoch 4/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 39.3918 - val_loss: 37.6336
Epoch 5/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 38.5394 - val_loss: 28.3231
Epoch 6/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 38.6326 - val_loss: 26.1029
Epoch 7/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 35.7899 - val_loss: 20.4708
Epoch 8/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 35.4328 - val_loss: 20.6410
Epoch 9/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 31.9183 - val_loss: 27.5206
Epoch 10/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 35.0059 - val_loss: 24.2268


## Evaluation

In [41]:
# sMAPE function
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))


In [42]:
# Make predictions
lstm_preds = lstm_model.predict(X_test)
cnn_preds = cnn_model.predict(X_test)
gru_preds = gru_model.predict(X_test)

126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [43]:
# Calculate sMAPE
lstm_smape = smape(y_test, lstm_preds)
cnn_smape = smape(y_test, cnn_preds)
gru_smape = smape(y_test, gru_preds)


In [44]:
# Display results
print(f"LSTM sMAPE: {lstm_smape:.2f}")
print(f"CNN sMAPE: {cnn_smape:.2f}")
print(f"GRU sMAPE: {gru_smape:.2f}")

LSTM sMAPE: 55.82
CNN sMAPE: 73.96
GRU sMAPE: 66.48


### new featuers

In [47]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Conv1D, Flatten

# List of features to include
features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load']
data = data[features]

# Train-Test Split to include all hourly data points in the last 7 days of each month
def train_test_split(data):
    # Get indices for the last 7 days of each month
    test_indices = data.index.to_series().groupby([data.index.year, data.index.month]).apply(lambda x: x[-24*7:])
    test_data = data.loc[test_indices]
    train_data = data.drop(test_indices)
    return train_data, test_data

# Perform the train-test split
train_data, test_data = train_test_split(data)

# Create Sequences for Time Series Data
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)  # Include all features in X
        y.append(data['price'].iloc[i+seq_length])  # Target is still electricity price
    return np.array(X), np.array(y)

# Set sequence length (e.g., 24 for daily pattern)
seq_length = 24
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

In [48]:
# Define Models

# LSTM Model
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, activation='relu', input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

# CNN Model
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        Flatten(),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

# GRU Model
def create_gru_model(input_shape):
    model = Sequential([
        GRU(50, activation='relu', input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

In [49]:
# Set input shape based on X_train
input_shape = (X_train.shape[1], X_train.shape[2])

In [50]:
# Train Models

# Train LSTM
lstm_model = create_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Train CNN
cnn_model = create_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Train GRU
gru_model = create_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 22859.3828 - val_loss: 1163.5002
Epoch 2/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1390.3772 - val_loss: 160.6864
Epoch 3/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 166.6660 - val_loss: 122.2878
Epoch 4/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 103.6810 - val_loss: 93.5798
Epoch 5/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 96.1220 - val_loss: 110.7205
Epoch 6/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 100.1979 - val_loss: 86.9600
Epoch 7/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 93.7845 - val_loss: 99.8928
Epoch 8/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 95.7831 - val_loss: 95.3826
Epoch 9/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 94.9677 - val_loss: 104.4508
Epoch 10/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 96.8288 - val_loss: 107.1328
Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1215.7247 - val_loss: 216.2246
Epoch 2/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - loss: 232.3160 - val_loss: 131.7258
Epoch 3/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - loss: 166.3294 - val_loss: 127.8805
Epoch 4/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - loss: 184.8764 - val_loss: 134.0547
Epoch 5/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - loss: 162.4510 - val_loss: 78.5899
Epoch 6/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 124.0528 - val_loss: 109.1081
Epoch 7/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - loss: 112.7699 - val_loss: 212.4310
Epoch 8/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 125.0139 - val_loss: 269.4325
Epoch 9/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 983us/step - loss: 113.2226 - val_loss: 56.0980
Epoch 10/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 98.3578 - val_loss: 194.5726
Epoch 1/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 438.6109 - val_loss: 97.1371
Epoch 2/10


In [51]:
# Evaluate and Compare Models Using sMAPE

# sMAPE function
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Make predictions
lstm_preds = lstm_model.predict(X_test)
cnn_preds = cnn_model.predict(X_test)
gru_preds = gru_model.predict(X_test)

# Calculate sMAPE
lstm_smape = smape(y_test, lstm_preds)
cnn_smape = smape(y_test, cnn_preds)
gru_smape = smape(y_test, gru_preds)

# Display results
print(f"LSTM sMAPE: {lstm_smape:.2f}")
print(f"CNN sMAPE: {cnn_smape:.2f}")
print(f"GRU sMAPE: {gru_smape:.2f}")

126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
LSTM sMAPE: 51.09
CNN sMAPE: 68.38
GRU sMAPE: 66.89


### updated

In [55]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Conv1D, Flatten, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import TimeSeriesSplit

# Load dataset
# data = pd.read_csv('your_data.csv', parse_dates=['date'], index_col='date')

# List of initial features
initial_features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load']
data = data[initial_features]

# Feature Engineering: Create Lagged Features and Rolling Statistics
def create_features(data):
    for lag in [1, 7, 24]:  # Lagged features for last hour, last 7 hours, and last 24 hours
        data[f'price_lag_{lag}'] = data['price'].shift(lag)
    data['price_rolling_mean_24'] = data['price'].rolling(window=24).mean()
    data['price_rolling_std_24'] = data['price'].rolling(window=24).std()
    return data

data = create_features(data)

# Cyclic Encoding for Time-Related Features
data['hour'] = data.index.hour
data['day_of_week'] = data.index.dayofweek
data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
data['day_of_week_sin'] = np.sin(2 * np.pi * data['day_of_week'] / 7)
data['day_of_week_cos'] = np.cos(2 * np.pi * data['day_of_week'] / 7)

# Final list of features to include
features = [
    'price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load', 
    'price_lag_1', 'price_lag_7', 'price_lag_24', 'price_rolling_mean_24', 'price_rolling_std_24',
    'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos'
]

data = data[features].dropna()  # Drop rows with NaN values due to lagged features

# 1. Train-Test Split
def train_test_split(data):
    test_indices = data.index.to_series().groupby([data.index.year, data.index.month]).apply(lambda x: x[-24*7:])
    test_data = data.loc[test_indices]
    train_data = data.drop(test_indices)
    return train_data, test_data

train_data, test_data = train_test_split(data)

# 2. Create Sequences for Time Series Data
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)  # Include all features in X
        y.append(data['price'].iloc[i+seq_length])  # Target is electricity price
    return np.array(X), np.array(y)

seq_length = 24
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)


In [56]:

# 3. Define Models with Dropout Regularization

# LSTM Model with Dropout
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
    return model

# CNN Model with Dropout
def create_cnn_model(input_shape):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        Flatten(),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
    return model

# GRU Model with Dropout
def create_gru_model(input_shape):
    model = Sequential([
        GRU(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
    return model

In [57]:
# Set input shape based on X_train
input_shape = (X_train.shape[1], X_train.shape[2])

# 4. Train and Evaluate Models Using Cross-Validation

# Initialize models
lstm_model = create_lstm_model(input_shape)
cnn_model = create_cnn_model(input_shape)
gru_model = create_gru_model(input_shape)

# Train and evaluate models
def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)
    preds = model.predict(X_test)
    return preds

lstm_preds = train_and_evaluate(lstm_model, X_train, y_train, X_test, y_test)
cnn_preds = train_and_evaluate(cnn_model, X_train, y_train, X_test, y_test)
gru_preds = train_and_evaluate(gru_model, X_train, y_train, X_test, y_test)

Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


378/378 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 13634.7773 - val_loss: 1377.4082
Epoch 2/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 3827.4893 - val_loss: 421.4138
Epoch 3/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 242.5427 - val_loss: 276.7422
Epoch 4/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 193.4321 - val_loss: 125.0931
Epoch 5/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 120.3142 - val_loss: 103.6696
Epoch 6/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 106.5658 - val_loss: 86.6853
Epoch 7/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 115.9406 - val_loss: 110.6955
Epoch 8/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 122.8999 - val_loss: 89.8546
Epoch 9/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 105.5241 - val_loss: 106.7652
Epoch 10/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 115.5681 - val_loss: 98.8850
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Epoch 1/10
378/378 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2410.323

In [58]:
# 5. Calculate sMAPE

# sMAPE function
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Calculate sMAPE for each model
lstm_smape = smape(y_test, lstm_preds)
cnn_smape = smape(y_test, cnn_preds)
gru_smape = smape(y_test, gru_preds)

# Display results
print(f"LSTM sMAPE: {lstm_smape:.2f}")
print(f"CNN sMAPE: {cnn_smape:.2f}")
print(f"GRU sMAPE: {gru_smape:.2f}")

LSTM sMAPE: 60.94
CNN sMAPE: 60.20
GRU sMAPE: 51.77


In [27]:
# Import libraries
import pywt
import pandas as pd

# Load original data
data = pd.read_pickle('../data/data.pkl') 
# data = pd.read_csv('your_data.csv', parse_dates=['date'], index_col='date')
features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load']

# Function to apply wavelet decomposition on a single time series
def wavelet_decompose(series, wavelet='db1', level=3):
    coeffs = pywt.wavedec(series, wavelet, level=level)
    decomposed_data = pd.DataFrame(coeffs).T
    decomposed_data.columns = [f'{series.name}_wavelet_{i}' for i in range(len(coeffs))]
    return decomposed_data

# Apply wavelet decomposition to each feature and store the decomposed components
decomposed_features = []
for feature in features:
    decomposed = wavelet_decompose(data[feature], wavelet='db1', level=3)
    decomposed.index = data.index[:len(decomposed)]  # Align indices with the original data
    decomposed_features.append(decomposed)

# Concatenate the decomposed features with the original data
decomposed_data = pd.concat(decomposed_features, axis=1)
data_with_wavelets = pd.concat([data, decomposed_data], axis=1)

# Drop rows with NaN values (these appear due to wavelet decomposition)
data_with_wavelets.dropna(inplace=True)


In [28]:
data.shape

(17520, 5)

In [29]:
decomposed_data.shape

(8760, 20)

In [30]:
data_with_wavelets.shape

(2190, 25)

In [32]:
# Import required libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error

# Load the wavelet-decomposed dataset (data_with_wavelets should be created from the previous steps)
# Make sure 'data_with_wavelets' includes the original and decomposed features
# data_with_wavelets = pd.read_csv('data_with_wavelets.csv', parse_dates=['date'], index_col='date')

# 1. Train-Test Split (keeping all hourly data points in the last 7 days of each month for testing)
def train_test_split(data):
    test_indices = data.index.to_series().groupby([data.index.year, data.index.month]).apply(lambda x: x[-24*7:])
    test_data = data.loc[test_indices]
    train_data = data.drop(test_indices)
    return train_data, test_data

train_data, test_data = train_test_split(data_with_wavelets)

# 2. Create Sequences for LSTM Model
# This function creates sequences of the specified length with all available features (original + decomposed)
def create_sequences(data, seq_length=24, target_column='price'):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)  # All features are used in X
        y.append(data[target_column].iloc[i+seq_length])  # Target is still the original 'price'
    return np.array(X), np.array(y)

# Set the sequence length (e.g., 24 for daily patterns)
seq_length = 24
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# 3. Define the LSTM Model using Decomposed Features
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

# Set input shape based on X_train (accounting for all original + decomposed features)
input_shape = (X_train.shape[1], X_train.shape[2])

# Initialize and train the model
lstm_model = create_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# 4. Evaluate the Model
# Define sMAPE function for evaluation
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Make predictions and calculate sMAPE
lstm_preds = lstm_model.predict(X_test)
lstm_smape = smape(y_test, lstm_preds)
print(f"LSTM sMAPE with Decomposed Features: {lstm_smape:.2f}")


Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 38566.1289 - val_loss: 23464.2500
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 20931.3125 - val_loss: 13564.5215
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 12160.7666 - val_loss: 13648.8916
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9905.8799 - val_loss: 8932.3047
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 6592.6714 - val_loss: 4137.1592
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 4248.2876 - val_loss: 3334.7336
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3031.0349 - val_loss: 1912.4199
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1969.6082 - val_loss: 1590.3502
Epoch 9/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1425.4758 - val_loss: 1126.7745
Epoch 10/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1256.4309 - val_loss: 1161.7040
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
LSTM sMAPE with Decomposed Features: 135.97
